In [1]:
# Twitter Sentiment Analysis


In [2]:
# download dependencies

#!pip install alpaca-py
#!pip install tweepy
#!pip install nltk


In [3]:
# Import dependencies

import re
import tweepy
import pandas as pd
from alpaca.data.live import CryptoDataStream
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [4]:
# Alpaca API Credentials
APCA_API_KEY = 'PKJNCK6G791HLT1JMRQV'
APCA_SECRET_KEY = 'W7COBHnWXOhA9aJlKdTfGjXyUiV14SrvwUwY2jQ2'
trading_client = TradingClient(APCA_API_KEY, APCA_SECRET_KEY, paper=True)
crypto_stream = CryptoDataStream(APCA_API_KEY, APCA_SECRET_KEY, raw_data=True)

# Twitter API Credentials
consumer_key = '9uZbJLS4xVltumTOS0SlnMghf'
consumer_secret = 'w2hC49P6LLRm1DtdCmdT6BCinM2YyAcSKpsJwraDqlaO8SOPkW'
access_token = '946010273839755264-eNJNnKdn08jfq2S7rVl9lFgUQ3nuL3e'
access_token_secret = '7j69ROBqlSMunJXu8W8XXJZtldWHMhXLmA2XT2umIR43m'
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# Define variables
num_tweets = 50000
keyword = 'BTC'
# keyword_to_asset = {
#    'BTC': 'BTCUSD'
#}

In [5]:
# Check whether account currently holds symbol
def check_positions(symbol):
    positions = trading_client.get_all_positions()
    if symbol in str(positions):
        return 1
    return 0
     

In [6]:
# Clean the tweet content using regex
def clean_tweet(tweet):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
     

In [7]:
clean_tweet

<function __main__.clean_tweet(tweet)>

In [8]:

# Retrieve tweets from Twitter using keyword
def get_tweets(query, count):
    tweets = set()
    fetched_tweets = api.search(q=query, count=count)
    for tweet in fetched_tweets:
        cleaned_tweet = clean_tweet(tweet.text)
        if cleaned_tweet not in tweets:
            tweets.add(cleaned_tweet)
    return tweets

In [9]:
# Calculating the polarity of each tweet using nltk
def calculate_polarity(tweets):
    scores = []
    for tweet in tweets:
        pol_score = SentimentIntensityAnalyzer().polarity_scores(tweet)
        pol_score['tweet'] = tweet
        scores.append(pol_score)
    return scores
     

In [10]:

# Placing trades based on the polarity of the tweets
def twitter_bot(symbol, close, qty=10):
    position = check_positions(symbol=symbol)
    tweets = get_tweets(keyword, num_tweets)
    scores = calculate_polarity(tweets)

    mean = pd.DataFrame.from_records(scores).mean()
    compound_score = mean['compound']
    print (f"Sentiment score: {round(compound_score, 3)}")

    if compound_score >= 0.05 and position==0:
        market_order_data = MarketOrderRequest(
            symbol=symbol,
            qty=qty,
            side=OrderSide.BUY,
            time_in_force=TimeInForce.GTC)

        trading_client.submit_order(
            order_data=market_order_data)
        print(f"Bought {symbol} at approx. {close}")

    elif compound_score <= -0.05 and position==1:
        trading_client.close_position(symbol_or_asset_id=symbol)
        print(f"Sold {symbol} at approx. {close}")
    return True

In [11]:
# Live streaming of crypto pricing data
async def quote_data_handler(data):
    close = data['c']
    twitter_bot(keyword_to_asset[keyword], close, qty=10)

crypto_stream.subscribe_bars(quote_data_handler, keyword_to_asset[keyword])
crypto_stream.run()

NameError: name 'keyword_to_asset' is not defined